## Periscope Table/View Dependency Tree

Following Periscope tutorial: https://www.sisense.com/blog/automated-identification-and-graphing-of-sql-dependencies/


### Pseudo Code

1. Clone the Periscope Repository
2. Iterate over the Files with a .sql extension
3. Create a dictionary/table/csv with: id, category (view, table, chart), code
3. For each file ending with a .sql extension, write the filename, category, and first word between '' and after 'FROM' or 'JOIN'


Packages: 

os—used to access  the current working directory

argparse—parses command line arguments

re—regex library for parsing out table names from SQL

csv—a library for navigating CSV files

networkx—modern Python graphviz library used for both creating and visualizing the directed acyclical graph

In [2]:
import os
home = os.path.expanduser("~")

In [29]:
cd ~/Desktop/periscope

/home/lucia/Desktop/periscope


In [17]:
# path = "./home/lucia/Desktop/periscope"
# for root,d_names,f_names in os.walk(path):
# 	print(root, d_names, f_names)


In [28]:
# for dirpath, dirs, files in os.walk("."):	 
# 	path = dirpath.split('/')
# 	print( (len(path))*'---', '[',os.path.basename(dirpath),']')
# 	for f in files:
# 		print(len(path)*'---', f)

In [52]:
query_map = {}

for dirpath, dirs, files in os.walk("./views/"):	
    for filename in files:
        if filename.endswith(".sql"):
            fname = os.path.join(dirpath,filename)
            with open(fname) as myfile:
                query_map[os.path.basename(fname).split(".")[0]] = myfile.read()

In [54]:
query_map["helper_avg_customers_month"]

'WITH temp AS(\n  SELECT q.time, SUM(cw) OVER (ORDER BY q.time ASC rows between unbounded preceding and current row) AS no_customers\n  FROM (\n    SELECT [dataset_customers.user_created_at:month] AS time, COUNT(DISTINCT dataset_customers.id) AS cw\n    FROM [dataset_customers]\n    GROUP BY 1) q)\n\nSELECT t1.time, (t1.no_customers+t2.no_customers)/2 as avgc\nFROM (SELECT [DATEADD(month,-1,time):month] AS time, no_customers FROM temp) t1\nJOIN temp t2 ON t1.time = t2.time'

In [58]:
#len(query_map)

# for key in query_map:
#     print(key)

In [59]:
import csv

In [89]:
csv_columns = ['Name', 'SQL']

In [96]:
# try:
#     with open('tree_mapping.csv', 'w') as csvfile:
#         writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#         writer.writeheader()
#         for data in query_map:
#             writer.writerow(data)
# except IOError:
#     print("I/O error")

In [95]:
! pip install pandas
import pandas as pd

     |████████████████████████████████| 10.0 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 20.2 MB 2.0 MB/s eta 0:00:01
  Using cached pytz-2019.3-py2.py3-none-any.whl (509 kB)


In [120]:
tree_df = pd.DataFrame(list(query_map.items()), columns=['name', 'sql'])
tree_df['type'] = 'view'
tree_df

,name,sql,type
0,view_cumulative_reserves,"--yes_cahce\nselect\n vc.user_id\n , month\n...",view
1,sessions_per_period,"(SELECT \n [created_at:aggregation] as date,\...",view
2,sepa_debit_average_runtime,"SELECT \n--charges.captured_at, charges.succes...",view
3,view_claims_metrics,--yes_cache\nwith reserve_data as (\n select\...,view
4,view_business_commissions,--yes_cache\nwith list_of_num as (select row_n...,view
...,...,...,...
136,view_expired_credits_global,--yes_cache\nwith received_credits as (\n sel...,view
137,view_cac,with\n helper_date as (\n select\n [d...,view
138,helper_basic_upgrade_modules_link,select\n upgrades.insurance_chain_id upgrade_...,view
139,helper_user_flow_type,--yes_cache \nselect \n shared_flow_session_i...,view


In [121]:
for row in tree_df:
    object_name = tree_df.name
    object_type = tree_df.type
    sql = tree_df.sql

Before we dig into the details, let’s review what we want to accomplish:

SQL cleanup

Lowercase everything

Remove block comments

Remove line comments

Replace all whitespace with single spaces

Get set of parents

Get set of Common Table Expressions(CTEs / with statement)

Remove CTEs from set of parents

Remove sub-selects

Clean up brackets*

*Sisense uses bracketed table names for views defined in the product; e.g., [customers_view].

In [123]:
#To achieve this, we define a set of RegEx strings:


import re


REG_BLOCK_COMMENT = re.compile("(/\*)[\w\W]*?(\*/)", re.I)
REG_LINE_COMMENT = re.compile('(--.*)', re.I)
REG_BRACKETS = re.compile("\[|\]|\)|\"", re.I)
REG_PARENTS = re.compile("(?<=join\s)+[\S\.\"\']+|(?<=from\s)+[\S\.\"\']+", re.I)
REG_CTES = re.compile("(\S+)\sas\W*\(", re.I)

In [124]:
# SQL cleanup

def clean_sql(sql):
   c_sql = sql.lower()  # lowercase everything (for easier match)
   c_sql = REG_BLOCK_COMMENT.sub('', c_sql)  # remove block comments
   c_sql = REG_LINE_COMMENT.sub('', c_sql)  # remove line comments
   c_sql = ' '.join(c_sql.split())  # replace \n and multi space w space
   return c_sql

c_sql = ‘ ‘.join(c_sql.split())

is a little tricky. This bit splits the SQL statement into an array based on arbitrary whitespace as a delimiter, and rejoins that array with a single space as a delimiter. This is to reduce multi-spaces, line breaks, and combinations of the two down to a single space.

### Extract Parents From SQL


In [148]:
# this returns the unique set of parents per query
def get_parents(c_sql):
   parents = set(REG_PARENTS.findall(c_sql))  
   return parents
# this returns the unique set of ctes per query, so we can exclude them from the list of parents
def get_ctes(c_sql):
   ctes = set(REG_CTES.findall(c_sql))  
   return ctes

In [150]:
  # clean the sql
def get_node_info(tree_df):
    
    node_info_dict = {}
    parent_dict = {}

    for sql in tree_df:

       c_sql = clean_sql(sql)
       # get the set of parents
       parents = get_parents(c_sql)
       # get set of ctes to exclude from parents
       ctes = get_ctes(c_sql)
       # remove CTES from parent dict
       for cte in ctes:
           parents.discard(cte)
       # get rid of brackets in views
       c_parents = set()
       for parent in parents:
           if not parent[:1] == '(':
               c_parents.add(REG_BRACKETS.sub('', parent))
       # add the object name and type and direct parents to the dict
       node_info_dict[object_name] = object_type  
       parent_dict[object_name] = c_parents
    return (parent_dict, node_info_dict)

In [139]:
# Define Relevant Node-set

def get_node_set(parent_dict, focal_node=None, direction=None):
 descendant_dict = {}  # intended to store all descendants (any generation)
 ancestor_dict = {}  # intended to store all ancestry (any generation)
 node_set = set()  # final result is stored and then returned via this set
 node_set.add(focal_node)

In [140]:
# this reverses a parent tree to a child tree
def get_child_dict(parent_dict):
   child_dict = {}
   for node in parent_dict:
       for parent in parent_dict[node]:
           if not parent in child_dict.keys():
               child_dict[parent] = set(node)
           else:
               child_dict[parent].add(node)
   return child_dict

In [141]:
 child_dict = get_child_dict(parent_dict)  # immediate children


In [142]:
child_dict

{}

In [143]:
 # build descendant dict
   for node in child_dict:
       descendant_dict[node] = traverse_tree(node, child_dict,
               been_done=set())
   # build ancestor dict
   for node in parent_dict:
       ancestor_dict[node] = traverse_tree(node, parent_dict,
               been_done=set())

In [144]:
def build_d_graph(parent_dict, node_set, node_type_dict):
   G = nx.DiGraph()  # initialize graph object
   for node in node_set:
       # Add color nodes
       if node_type_dict.get(node, None) == 'view':
           G.add_node(node, color='green')
       elif node_type_dict.get(node, None) == 'chart':
           G.add_node(node, color='blue')
       elif node_type_dict.get(node, None) == 'csv':
           G.add_node(node, color='red')
       # add edges and non-color nodes
       for parent in parent_dict.get(node, set()):
           G.add_edge(parent, node)
   return G

In [153]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
